In [99]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn

import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler,Normalizer

from tqdm import tqdm

import matplotlib.pyplot as plt

In [100]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [101]:
df = pd.read_csv('train_data.csv',index_col='Unnamed: 0')

In [102]:
df.head()

,t2m,NDVI,boundaryLayerHeight,rh,totalPrecipitation,windSpeed,windDirection,SWIR1,NIR,red,...,day_sin,season_cos,season_sin,week_cos,week_sin,month_cos,month_sin,Grasses_avg_daily,Grasses_std_daily,Grasses
0,-0.369523,0.4982,102.408676,88.53255,0.000732,0.649779,-135.56980,7903.0,8511.0,2964.0,...,0.017213,6.123234e-17,1.0,1.0,-2.449294e-16,0.866025,0.5,0.0,0.0,0.0
1,0.845408,0.4982,167.713550,94.78888,0.001186,1.490278,-153.75153,7482.0,6773.0,3226.0,...,0.017213,6.123234e-17,1.0,1.0,-2.449294e-16,0.866025,0.5,0.0,0.0,0.0
2,-3.842715,0.4982,62.996920,84.02357,0.000781,0.823700,162.49753,6292.0,7443.0,2565.0,...,0.017213,6.123234e-17,1.0,1.0,-2.449294e-16,0.866025,0.5,0.0,0.0,0.0
3,0.307328,0.4982,165.393590,96.01055,0.000915,1.141481,-117.63472,9590.0,10635.0,3852.0,...,0.017213,6.123234e-17,1.0,1.0,-2.449294e-16,0.866025,0.5,0.0,0.0,0.0
4,-11.214022,0.4982,27.412760,83.98948,0.000880,0.661277,156.37183,5678.0,8548.0,1672.0,...,0.017213,6.123234e-17,1.0,1.0,-2.449294e-16,0.866025,0.5,0.0,0.0,0.0


In [103]:
class MyDataset(Dataset):
    def __init__(self,dataset,transform=None):
        # data loading
        X = dataset.iloc[:,:-1].values.astype('float32')
        self.y = dataset.iloc[:,-1].values.reshape(-1,1).astype('float32')
        self.n_samples = dataset.shape[0]
        
        # scaling the features
        scaler = StandardScaler()
        self.X_norm = scaler.fit_transform(X)
        
        self.transform = transform

    def __len__(self):
        return self.n_samples
        
    def __getitem__(self,index):
        sample = self.X_norm[index],self.y[index]
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [104]:
class ToTensor:
    def __call__(self,sample):
        inputs,targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

In [105]:
composed = transforms.Compose([ToTensor()])

In [106]:
dataset = MyDataset(df,transform=composed)

In [107]:
feature, label = dataset[0]
feature.shape, label.shape

(torch.Size([38]), torch.Size([1]))

In [136]:
# Constants
input_size = 38
hidden_size = 4
output_size = 1
learning_rate = 0.00001
n_epochs = 50
batch_size = 100

In [137]:
train_loader = DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True)

In [138]:
# Model
class PollenModel(nn.Module):
    def __init__(self,input_dim,hidden_size,output_dim):
        super(PollenModel,self).__init__()
        
        self.pollen = nn.Sequential(
                                    nn.Linear(input_dim,hidden_size),
                                    nn.ReLU(inplace=True),
                                    nn.Linear(hidden_size,output_dim)
                                    )
    def forward(self,x):
        return self.pollen(x)

In [139]:
model = PollenModel(input_size,hidden_size,output_size)

In [140]:
# Loss & Optemizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [141]:
n_total_steps = len(train_loader)

for epoch in range(n_epochs):
    for i, (features,value) in enumerate(tqdm(train_loader)):
        
        features = features.reshape(-1,38).to(device)
        value = value.to(device)
        
        model = model.to(device)
        criterion = criterion.to(device)
    
        # Predict
        y_pred = model(features)
        # Compare
        loss = torch.sqrt(criterion(y_pred,value))
        # Learn
        dw = loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if (i+1) % 1023 ==0:
            print(f'epoch {epoch+1} / {n_epochs}, loss = {loss.item():.4f}')

100%|█████████████████████████████████████| 1023/1023 [00:02<00:00, 449.86it/s]


epoch 1 / 50, loss = 40.9829


100%|█████████████████████████████████████| 1023/1023 [00:02<00:00, 453.11it/s]


epoch 2 / 50, loss = 39.2096


100%|█████████████████████████████████████| 1023/1023 [00:02<00:00, 447.31it/s]


...

epoch 48 / 50, loss = 55.7244


100%|█████████████████████████████████████| 1023/1023 [00:02<00:00, 443.95it/s]


epoch 49 / 50, loss = 71.3639


100%|█████████████████████████████████████| 1023/1023 [00:02<00:00, 447.99it/s]

epoch 50 / 50, loss = 25.2449


In [142]:
with torch.no_grad():
    list_corr = []
    list_pred = []
    for features,value in train_loader:
        
        features = features.reshape(-1,38).to(device)
        value = value.cpu().numpy()
    #     y_pred = model(features)
    #     predict = y_pred.cpu().numpy()
    #     list_corr.append(value)
    #     list_pred.append(predict)
    # predict = np.concatenate(list_pred)
    # value = np.concatenate(list_corr)
    # plt.scatter(predict,value)
    # plt.show()
    sys.exit()

NameError: name 'sys' is not defined

In [146]:
y_pred = model(features[0])

In [150]:
value[0],y_pred

(array([0.], dtype=float32),
 tensor([-0.5983], device='cuda:0', grad_fn=<AddBackward0>))

In [148]:
value

array([[  0.],
       [  1.],
       [  0.],
       [ 42.],
       [  0.],
       [  0.],
       [  4.],
       [  0.],
       [  0.],
       [  0.],
       [ 34.],
       [  0.],
       [  0.],
       [  0.],
       [  2.],
       [ 32.],
       [  3.],
       [  8.],
       [  0.],
       [ 14.],
       [  0.],
       [  0.],
       [  0.],
       [  2.],
       [  4.],
       [  6.],
       [  0.],
       [  0.],
       [  4.],
       [  0.],
       [  0.],
       [  0.],
       [  6.],
       [  2.],
       [  1.],
       [  0.],
       [  0.],
       [ 18.],
       [ 62.],
       [  9.],
       [  0.],
       [  6.],
       [  0.],
       [  0.],
       [  0.],
       [282.],
       [  0.],
       [  0.],
       [ 20.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [ 38.],
       [  0.],
       [  0.],
       [  4.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  